# INSTRUCTOR: Pub/Sub Lab Management

This notebook is for instructors only. It provides tools for:
1. **Provisioning** — Create/teardown all AWS resources
2. **Monitoring** — Check queue depths and message flow
3. **Simulator** — Populate queues with multi-host events (Section 5)
4. **Chaos** — Inject duplicates, spikes, and late events (Section 6)
5. **Reset** — Drain all queues between sessions

In [ ]:
!pip install -q boto3

In [ ]:
import os
import json
import boto3
import uuid
import time
import random
from datetime import datetime, timezone, timedelta
from collections import Counter

# --- Instructor credentials ---
os.environ["AWS_ACCESS_KEY_ID"]     = ""  # your access key
os.environ["AWS_SECRET_ACCESS_KEY"] = ""  # your secret key
os.environ["AWS_DEFAULT_REGION"]    = "us-east-1"

sns = boto3.client("sns", region_name="us-east-1")
sqs = boto3.client("sqs", region_name="us-east-1")

print("Connected to AWS.")

## Configuration

All resource names and student list. Edit as needed.

In [ ]:
REGION = "us-east-1"
ACCOUNT_ID = "194722398367"
PREFIX = "ds2032"

STUDENTS = ["bilge", "sam", "phin", "manuel", "bruno", "gil"]

TOPIC_NAME = f"{PREFIX}-view-events.fifo"
TOPIC_ARN = f"arn:aws:sns:{REGION}:{ACCOUNT_ID}:{TOPIC_NAME}"

STUDENT_QUEUES = {
    s: f"https://sqs.{REGION}.amazonaws.com/{ACCOUNT_ID}/{PREFIX}-node-{s}.fifo"
    for s in STUDENTS
}

AUDITOR_QUEUES = {
    f"auditor-{x}": f"https://sqs.{REGION}.amazonaws.com/{ACCOUNT_ID}/{PREFIX}-auditor-{x}.fifo"
    for x in ["a", "b", "c"]
}

CHAOS_TOPIC_ARN = f"arn:aws:sns:{REGION}:{ACCOUNT_ID}:{PREFIX}-view-events-chaos"
CHAOS_QUEUE_URL = f"https://sqs.{REGION}.amazonaws.com/{ACCOUNT_ID}/{PREFIX}-chaos-observer"

FIFO_GROUP = "view-events"

ALL_QUEUES = {**STUDENT_QUEUES, **AUDITOR_QUEUES, "chaos": CHAOS_QUEUE_URL}

print(f"Configured for {len(STUDENTS)} students.")
print(f"Topic: {TOPIC_ARN}")

## Monitoring

Check how many messages are in each queue.

In [ ]:
# Check queue depths

print(f"{'Queue':45s} {'Available':>10s} {'In Flight':>10s}")
print("-" * 70)

for name, url in ALL_QUEUES.items():
    try:
        attrs = sqs.get_queue_attributes(
            QueueUrl=url,
            AttributeNames=["ApproximateNumberOfMessages", "ApproximateNumberOfMessagesNotVisible"]
        )["Attributes"]
        avail = attrs.get("ApproximateNumberOfMessages", "?")
        inflight = attrs.get("ApproximateNumberOfMessagesNotVisible", "?")
        print(f"  {name:43s} {avail:>10s} {inflight:>10s}")
    except Exception as e:
        print(f"  {name:43s} ERROR: {e}")

## Reset

Drain all queues. Use between class sessions.

In [ ]:
# Drain ALL queues

print("Draining all queues...\n")
total = 0
for name, url in ALL_QUEUES.items():
    count = 0
    while True:
        resp = sqs.receive_message(QueueUrl=url, MaxNumberOfMessages=10, WaitTimeSeconds=1)
        msgs = resp.get("Messages", [])
        if not msgs:
            break
        for m in msgs:
            sqs.delete_message(QueueUrl=url, ReceiptHandle=m["ReceiptHandle"])
            count += 1
    if count:
        print(f"  {name}: drained {count}")
    total += count

print(f"\nTotal drained: {total}")

## Simulator (Section 5)

Run this BEFORE Section 5. Publishes events from multiple simulated hosts.
Students will see mixed traffic in their queues.

In [ ]:
# --- Simulator Configuration ---
NUM_HOSTS = 3        # Number of simulated hosts
NUM_EVENTS = 30      # Total events to publish
RATE = "fast"        # "slow", "normal", or "fast"

FAKE_HOSTS = ["host-alpha", "host-beta", "host-gamma", "host-delta", "host-epsilon"][:NUM_HOSTS]
CONTENT = ["video-matrix", "article-ai-future", "video-matrix-reloaded",
           "podcast-distributed-sys", "article-quantum-2032", "video-crypto-explained"]
ADS = ["ad-pepsi", "ad-nvidia", "ad-coca-cola-2032", "ad-aws", "ad-google"]

In [ ]:
# Run simulator

print(f"Publishing {NUM_EVENTS} events from {FAKE_HOSTS}...\n")
events_by_host = {h: 0 for h in FAKE_HOSTS}

for i in range(NUM_EVENTS):
    host = random.choice(FAKE_HOSTS)
    event = {
        "event_id": str(uuid.uuid4()),
        "host_id": host,
        "content_id": random.choice(CONTENT),
        "ad_id": random.choice(ADS),
        "timestamp": datetime.now(timezone.utc).isoformat(),
    }
    sns.publish(TopicArn=TOPIC_ARN, Message=json.dumps(event), MessageGroupId=FIFO_GROUP)
    events_by_host[host] += 1
    print(f"  [{i+1:3d}] {host}: {event['content_id']}")

    if RATE == "normal": time.sleep(random.uniform(0.05, 0.2))
    elif RATE == "slow": time.sleep(random.uniform(0.5, 1.0))

print(f"\nSummary:")
for h, c in sorted(events_by_host.items()):
    print(f"  {h}: {c}")
print(f"  Total: {NUM_EVENTS}")

## Chaos Injection (Section 6)

Run this BEFORE Section 6. Injects problems into the **standard** chaos queue.
Students will detect duplicates, spikes, and late events.

In [ ]:
# Chaos: Duplicate Events

DUP_COUNT = 20
DUP_RATE = 0.3

print(f"Injecting {DUP_COUNT} events, ~{int(DUP_RATE*100)}% duplicates...\n")

events_pool = []
for i in range(DUP_COUNT):
    if events_pool and random.random() < DUP_RATE:
        event = random.choice(events_pool)
        print(f"  [{i+1:3d}] DUPLICATE: {event['event_id'][:12]}...")
    else:
        event = {
            "event_id": str(uuid.uuid4()),
            "host_id": random.choice(["host-alpha", "host-beta", "host-gamma"]),
            "content_id": random.choice(CONTENT),
            "ad_id": random.choice(ADS),
            "timestamp": datetime.now(timezone.utc).isoformat(),
        }
        events_pool.append(event)
        print(f"  [{i+1:3d}] NEW:       {event['event_id'][:12]}...")

    sns.publish(TopicArn=CHAOS_TOPIC_ARN, Message=json.dumps(event))

print(f"\nDone. Duplicates will appear in the chaos queue.")

In [ ]:
# Chaos: Volume Spike

SPIKE_HOST = "host-SUSPICIOUS"
SPIKE_COUNT = 50

print(f"Flooding {SPIKE_COUNT} events from '{SPIKE_HOST}'...\n")

for i in range(SPIKE_COUNT):
    event = {
        "event_id": str(uuid.uuid4()),
        "host_id": SPIKE_HOST,
        "content_id": "video-fake-content",
        "ad_id": "ad-fake-advertiser",
        "timestamp": datetime.now(timezone.utc).isoformat(),
    }
    sns.publish(TopicArn=CHAOS_TOPIC_ARN, Message=json.dumps(event))
    if (i + 1) % 20 == 0:
        print(f"  Published {i+1}/{SPIKE_COUNT}")

print(f"\nSpike complete.")

In [ ]:
# Chaos: Late Events

LATE_COUNT = 10
LATE_MINUTES = 30

print(f"Injecting {LATE_COUNT} events with timestamps {LATE_MINUTES} min in the past...\n")

for i in range(LATE_COUNT):
    mins_ago = LATE_MINUTES + random.randint(-5, 5)
    fake_ts = (datetime.now(timezone.utc) - timedelta(minutes=mins_ago)).isoformat()
    event = {
        "event_id": str(uuid.uuid4()),
        "host_id": random.choice(["host-alpha", "host-beta"]),
        "content_id": "video-matrix",
        "ad_id": "ad-pepsi",
        "timestamp": fake_ts,
    }
    sns.publish(TopicArn=CHAOS_TOPIC_ARN, Message=json.dumps(event))
    print(f"  [{i+1}] claimed {mins_ago} min ago")

print(f"\nLate events injected.")